<a href="https://colab.research.google.com/github/hebaashraf21/KNN_K-means_PCA_CUDA_Implementation/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Setup cuda environment
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7y0yaj8_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7y0yaj8_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-4l9fjsyf/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpstj_e3yu".


In [13]:
import time
import numpy as np
from numba import cuda


def knn_without_cuda(reference_points, query_points, k):
    results = []
    for query_point in query_points:
        distances = np.linalg.norm(reference_points - query_point, axis=1)
        nearest_indices = np.argsort(distances)[:k]
        results.append(nearest_indices)
    return results



# Generate sample data
reference_points = np.random.rand(100000, 3)
query_points = np.random.rand(10000, 3)
k = 5

# Measure time for non-CUDA implementation
start_time = time.time()
knn_without_cuda(reference_points, query_points, k)
end_time = time.time()
print("Time taken without CUDA:", end_time - start_time, "seconds")



Time taken without CUDA: 128.67473578453064 seconds


In [14]:
%%writefile knn.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <cuda_runtime.h>

#define MAX_ERR 1e-6

__global__ void knn_kernel(float *reference_points, float *query_points, int *results, int n, int m, int k, int dim) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < m) {
        float min_distance = INFINITY;
        int min_index = -1;
        for (int i = 0; i < n; ++i) {
            float distance = 0;
            for (int j = 0; j < dim; ++j) {
                float diff = reference_points[i * dim + j] - query_points[idx * dim + j];
                distance += diff * diff;
            }
            if (distance < min_distance) {
                min_distance = distance;
                min_index = i;
            }
        }
        results[idx * k] = min_index; //Storing only the nearest reference point
    }
}

void knn_with_cuda(float *reference_points, float *query_points, int *results, int n, int m, int k, int dim) {
    float *d_reference, *d_query;
    int *d_results;

    cudaMalloc((void **)&d_reference, sizeof(float) * dim * n);
    cudaMalloc((void **)&d_query, sizeof(float) * dim * m);
    cudaMalloc((void **)&d_results, sizeof(int) * m * k);

    cudaMemcpy(d_reference, reference_points, sizeof(float) * dim * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_query, query_points, sizeof(float) * dim * m, cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (m + blockSize - 1) / blockSize;

    knn_kernel<<<numBlocks, blockSize>>>(d_reference, d_query, d_results, n, m, k, dim);

    cudaMemcpy(results, d_results, sizeof(int) * m * k, cudaMemcpyDeviceToHost);

    cudaFree(d_reference);
    cudaFree(d_query);
    cudaFree(d_results);
}

int main() {
    int n = 100000; // Number of reference points
    int m = 10000;  // Number of query points
    int k = 5;      // Number of nearest neighbors
    int dim = 3;    // Dimensionality of points

    float *reference_points = (float *)malloc(sizeof(float) * dim * n);
    float *query_points = (float *)malloc(sizeof(float) * dim * m);
    int *results = (int *)malloc(sizeof(int) * m * k);

    // Initialize reference_points and query_points with random values
    for (int i = 0; i < dim * n; ++i) {
        reference_points[i] = static_cast<float>(rand()) / RAND_MAX;
    }
    for (int i = 0; i < dim * m; ++i) {
        query_points[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    knn_with_cuda(reference_points, query_points, results, n, m, k, dim);
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Time taken with CUDA: %f milliseconds\n", milliseconds);

    free(reference_points);
    free(query_points);
    free(results);

    return 0;
}


Overwriting knn.cu


In [15]:
!nvcc knn.cu -o knn
!./knn


Time taken with CUDA: 102.770851 milliseconds
